### **Online Knowledge Distillation with Diverse Peers**

#### **Abstract**

- KD 并不总是可用
- **组派生**的目标(*group-derived targets*)为无教师蒸馏提供了良好的配方，但组成员通过简单的聚合函数**快速均质(*homogenized*)，导致早期饱和**
- *Online Knowledge Distillation with Diverse peers(OKDDip), which performs two-level distillation during training with multiple auxiliary peers and one group leader.*
    - **In the first-level distillation**, each auxiliary peer holds an individual set of **aggregation weights generated with an attention-based mechanism** to derive its own targets from predictions of other auxiliary peers.
    - **The second-level distillation** is performed to transfer the knowledge in the ensemble of auxiliary peers further **to the group leader**, i.e., the model used for inference
- 在不牺牲训练和推理性能的情况下，**OKDDip 效果比最先进的方法还要好**

------------------------------
#### **Introduction**
- Since the quality of predictions varies among peers, **it is important to treat peers unequally** (Lan, Zhu, and Gong 2018).
  - Unfortunately, naive aggregation functions tend to **cause peers to quickly homogenize, hurting the effectiveness of group distillation** (Kuncheva and Whitaker 2003; Zhou 2012).

- Unlike naive group-based learning where all peers end up with similar behaviors, **trained peer models in our approach could be quite different from each other**.

- **A key design** of OKDDip is that each auxiliary peer assigns individual weights to all the peers during aggregation **to derive their own target distributions**. We incorporate an **attention-based mechanism** (Vaswani et al. 2017) to generate a distinct set of weights **for each peer** to measure the importance of group members. This allows large variation in derived target distributions and hence boosts peer diversity. **Note that asymmetric weights exist in our model**.

- dataset: CIFAR-10, CIFAR-100, ImageNet-2012

--------------------------------
#### **Related Work**
- **Knowledge Distillation**
- **Online Knowledge Distillation**
    - **The main difference in these approaches is the way that each student model learns from others.**
- **Self-Attention**


-----------------------------------------
#### **Online Knowledge Distillation with Diverse Peers**
- $T = 3$, 相对权重设置为 $T^2$

![Figure 1: An overview of the proposed Online Knowledge Distillation with Diverse Peers(OKDDip).](../img/Figure-1_An-overview-of-the-proposed-Online-Knowledge-Distillation-with-Diverse-Peers_OKDDip.png)

##### **Loss function**
$$\mathcal{L}_{OKDDip} = \sum_{a=1}^{m} \mathcal{L}_{gt}(a) + T^2 \mathcal{L}_{dis1} + T^2 \mathcal{L}_{dis2}$$

- 其中第一项为所有的 *student model* 和 *ground-truth* 的交叉熵损失求和

- *For the first-level distillation, each auxiliary peer for $a = 1, 2, . . . m − 1$ distills from **its own group-derived soft targets $\mathbf{t_a}$**, which is computed by aggregating predictions of all peers with different weights*
$$\mathbf{t_a} = \sum_{b=1}^{m-1} \alpha_{ab} \mathbf{q_b^`}$$

*where $α_{ab}$ represents the extent to which the $bth$ member is attended in derivation $t_a$, and $\sum_b \alpha_{ab} = 1$. The distillation loss of all auxiliary peers is then given as*
$$\mathcal{L}_{dis1} = \sum_{a=1}^{m-1} \mathrm{KL} (\mathbf{t_a}, \mathbf{q_a^`})$$

*which could be regarded as **weighted regularization for the output distribution**. As pointed in (Pereyra et al. 2017), penalizing the confident prediction could prevent over-fitting by increasing the probabilities assigned to incorrect classes.*

> **简而言之，$\mathcal{L}_{dis1}$ 实现了个性化定制 teacher model，并一对一教学**

- *The group knowledge of those auxiliary peers is distilled further **to the group leader** (i.e., the mth student model) with the second distillation*
$$\mathcal{L}_{dis2} = \mathrm{KL} (\mathbf{t_m}, \mathbf{q_m^`})$$

> **With** diversity enhancement by the first-level distillation, we simply **average** the predictions of all auxiliary peers to compute tm.

> Following (Lan, Zhu, and Gong 2018; Laine and Aila 2017), the two distillation terms would be multiplied by an iterationdependent weighting function during implementation to **avoid large contribution of distillation in early stages.**

**也就是说不同时期的 *group leader* 可以有不同的计算方式**

##### **Attention-based weights**
- 不同的初始化
- *Project the extracted features of each peer model $\mathbf{h_a}$ into two subspaces separately by linear transformation*
$$L(\mathbf{h_a}) = \mathbf{W}_L^T \mathbf{h}_a \qquad \text{and} \qquad E(\mathbf{h_a}) = \mathbf{W}_E^T \mathbf{h}_a$$

*where $\mathbf{W}_L$ and $\mathbf{W}_E$ are the learned projection matrices **shared by all auxiliary peers**. Similar to self-attention, $\alpha_{ab}$ is calculated as Embedded Gaussian distance with normalization*
$$\alpha_{ab} = \frac{\exp(L(\mathbf{h_a})^T E(\mathbf{h_b}))}{\sum_{f=1}^{m-1} \exp(L(\mathbf{h_a})^T E(\mathbf{h_f}))}$$

> 我们可以将 $L(\mathbf{h_a})$ 和 $E(\mathbf{h_a})$ 分别看作 $Q$ 和 $K$

- *Weights generated in this way have the following merits:*
    - **Asymmetric**: *The asymmetric property provides a possible way to suppress negative effect in one direction without stopping positive guidance in the other, **which is important for mutual learning between two peers optimized to different levels**. It on the one hand reduces the extent to which a well-behaved model is affected by a poorly performed peer by assigning a small weight, and on the other hand allows the less optimized model to learn from the better optimized one with a large weight.*
    - **Dynamic**: *The performance of peer models changes during training, updating weights in iterations allows each model to attend a dynamic set of peers adaptively.*

> 也就是说，**非对称性给了 不太好的模型向好的模型去学习，好的模型不被不好的模型拉下水 的可能性**

##### **Why learning with distinct target distributions leads to diversity?**
详见原文

**Training and deployment**. *The proposed framework may be implemented with **branch-based or network-based** student models. **In a branch-based setting**, all student models share the first several layers to use the same low-level features, and separate from each other from a certain layer to have individual branches for high-level feature extraction and classification. **In a network-based setting**, student models are individual networks. All auxiliary peers are discarded after training and only the group leader is kept **for deployment**. There is no additional increase in complexity or cost compared to other group-based approaches given the number and architecture of student models are the same.*

--------------------------------------
#### **Experiments**
- *dataset: CIFAR-10, CIFAR-100, ImageNet-2012*
- *Newwork architecture: VGG-16, ResNet-32, ResNet-34, ResNet-110, WRN-28-8, and DenseNet-40-12*
- *[code](https://github.com/DefangChen/OKDDip-AAAI2020)*
- 具体参数设置见原文

---------------------------------
#### **Conclusion**